In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Dense, LSTM, TimeDistributed
from tensorflow.keras.models import Sequential
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import brown
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import brown, treebank
from tensorflow.keras.optimizers import Adam
from gensim.models import KeyedVectors
import gensim.downloader as api
from nltk.corpus import conll2000
import os
path = api.load("word2vec-google-news-300", return_path=True)
tf.config.run_functions_eagerly(True)

2023-03-25 13:35:11.859379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
nltk.download('treebank')
nltk.download('conll2000')

[nltk_data] Downloading package brown to /Users/annielin/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/annielin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/annielin/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
txt = brown.tagged_words(tagset='universal')
txt2 = treebank.tagged_words(tagset='universal')
brown_sent = brown.tagged_sents(tagset='universal')
tree_sent = treebank.tagged_sents(tagset='universal')
conll_sent = conll2000.tagged_sents(tagset='universal')
all_sent = brown_sent + tree_sent + conll_sent

In [11]:
# Tokenize the words

word = [word[0].lower() for tup in all_sent for word in tup] # store the word
pos = [pos[1].lower() for tup in all_sent for pos in tup] # store the corresponding pos tag

word_tokenizer = Tokenizer()
pos_tokenizer = Tokenizer()

word_tokenizer.fit_on_texts(word) 
word_seqs = word_tokenizer.texts_to_sequences(word)  
pos_tokenizer.fit_on_texts(pos)
pos_seqs = pos_tokenizer.texts_to_sequences(pos)


In [6]:
# find the length of the training sets
max_len = 1 # don't need to pad fixed len dataset
w_size = len(word_tokenizer.word_index) + 1
pos_size = len(pos_tokenizer.word_index) + 1


In [7]:
# code modified from https://towardsdatascience.com/pos-tagging-using-rnn-7f08a522f849
word_vectors = KeyedVectors.load_word2vec_format(path, binary=True)
embedding_size = 300  
embedding_weights = np.zeros((w_size, embedding_size))
word2id = word_tokenizer.word_index
for word, index in word2id.items():
    try:
        embedding_weights[index, :] = word_vectors[word]
    except KeyError:
        pass

In [8]:
# the LSTM Model
model = Sequential()
optimizer = Adam(learning_rate=0.001)

model.add(Embedding(w_size, embedding_size, input_length=max_len, weights=[embedding_weights], trainable=False)) 
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(pos_size, activation='softmax')))
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


2023-03-23 23:50:24.330887: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 300)          13362600  
                                                                 
 lstm (LSTM)                 (None, 100, 64)           93440     
                                                                 
 time_distributed (TimeDistr  (None, 100, 12)          780       
 ibuted)                                                         
                                                                 
Total params: 13,456,820
Trainable params: 94,220
Non-trainable params: 13,362,600
_________________________________________________________________
None


In [10]:
# training data

# split the data for better training

split_idx = int(0.8 * w_size)
word_train, pos_train = word_seqs[:split_idx], pos_seqs[:split_idx]
word_test, pos_test = word_seqs[split_idx:], pos_seqs[split_idx:]

# pad the data
# word_train = pad_sequences(word_train, max_len, padding='post', truncating='post') 
# pos_train = pad_sequences(pos_train, max_len, padding='post', truncating='post')
# word_test = pad_sequences(word_test, maxlen=max_len, padding='post', truncating='post')
# pos_test = pad_sequences(pos_test, maxlen=max_len, padding='post', truncating='post')

# convert pos data set into one-hot encoding
pos_train = to_categorical(pos_train, num_classes=pos_size)
pos_test = to_categorical(pos_test, num_classes=pos_size)

training = model.fit(word_train, pos_train, batch_size=32, epochs=10, validation_data=(word_test, pos_test))
loss, accuracy = model.evaluate(word_test, pos_test)
model.save('lstm_model.h5')
print('Model Done')

/Users/annielin/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


35174/35174 [==============================] - 4272s 121ms/step - loss: 0.0061 - accuracy: 0.9981
Model Done


In [ ]:
print(model.summary())

In [ ]:
#  right now the model is trained with only brown data set for faster runtime
#  may increase the data with treebank for better training in future